# Part 1

In [1]:
import random 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json 
!pip install geopy
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
#!pip install folium
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium
from bs4 import BeautifulSoup
!pip install html5lib
!pip install lxml

print('Everything imported.')

Everything imported.


Scrape the Wikipedia page https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M .
The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [2]:
df= pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header=0)
toronto_NH = df[0]
toronto_NH.columns = ['PostalCode','Borough','Neighborhood']
toronto_NH.head()

PostalCode           Borough               Neighborhood
0        M1A      Not assigned               Not assigned
1        M2A      Not assigned               Not assigned
2        M3A        North York                  Parkwoods
3        M4A        North York           Victoria Village
4        M5A  Downtown Toronto  Regent Park, Harbourfront

 Use the .shape method to print the number of rows of the original dataframe.

In [3]:
toronto_NH.shape

(180, 3)

Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. 
These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11  in the above table.

In [4]:
toronto_table = toronto_NH[toronto_NH.Borough != 'Not assigned'].reset_index(drop=True)
toronto_table.head()

PostalCode           Borough                                 Neighborhood
0        M3A        North York                                    Parkwoods
1        M4A        North York                             Victoria Village
2        M5A  Downtown Toronto                    Regent Park, Harbourfront
3        M6A        North York             Lawrence Manor, Lawrence Heights
4        M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government

If a cell has a borough but a "Not assigned"  neighborhood, then the neighborhood will be the same as the borough. 
But this does not apply.

In [5]:
Not_assigned = toronto_table['Neighborhood'] == "Not assigned"
toronto_table.loc[Not_assigned, 'Neighborhood'] = toronto_table.loc[Not_assigned, 'Borough']
toronto_table[Not_assigned]

Empty DataFrame
Columns: [PostalCode, Borough, Neighborhood]
Index: []

Here I used the .shape method to print the number of rows of my dataframe.

In [6]:
toronto_table.shape

(103, 3)

# Part 2

 Import the csv file that has the geographical coordinates of each postal code from https://cocl.us/Geospatial_data.
 I used the method .head() to have a look at the dataframe.

In [7]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_90e381095a1e4558b2df2cd2d4d19226 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='eSMR8kAO7DX0VwSuCoykiPq7xq5nxk8TbLP-xsulX6Cn',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_90e381095a1e4558b2df2cd2d4d19226.get_object(Bucket='egmentingandclusteringneighborhoo-donotdelete-pr-ujs6qrolst7nwm',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

Geospatial_Coordinates = pd.read_csv(body)
Geospatial_Coordinates.head()

Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476

I renamed the column "Postal Code" to "PostalCode" so that they are named identical.
Then I merged the 2 tables and used the method .head() to have a look at the dataframe.

In [8]:
Geospatial_Coordinates.rename(index=str, columns={"Postal Code": "PostalCode"}, inplace = True)
toronto_table_merged = pd.merge(toronto_table, Geospatial_Coordinates, on='PostalCode', how='inner')
toronto_table_merged.head()

PostalCode           Borough                                 Neighborhood  \
0        M3A        North York                                    Parkwoods   
1        M4A        North York                             Victoria Village   
2        M5A  Downtown Toronto                    Regent Park, Harbourfront   
3        M6A        North York             Lawrence Manor, Lawrence Heights   
4        M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   

    Latitude  Longitude  
0  43.753259 -79.329656  
1  43.725882 -79.315572  
2  43.654260 -79.360636  
3  43.718518 -79.464763  
4  43.662301 -79.389494

# Part 3

In order to define an instance of the geocoder, I need to define a user_agent. I have named my agent 'totonto'.
Afterwards I get the coordinates of Toronto and display them.

In [9]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


Now I plot the date into the toronto_map. First I create map of Toronto using latitude and longitude values, thebn I add the markers to the map.

In [10]:
# Explore and cluster the neighborhoods in Toronto
toronto_map= folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_table_merged['Latitude'], toronto_table_merged['Longitude'], toronto_table_merged['Borough'], toronto_table_merged['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map) 
toronto_map

For illustration purposes, I simplify the above map and segment and cluster only the neighborhoods in North York. So I slice the original dataframe and create a new dataframe of the North York data.

In [11]:
NorthYork_data = toronto_table_merged[toronto_table_merged['Borough'] == 'North York'].reset_index(drop=True)
NorthYork_data.head()

PostalCode     Borough                      Neighborhood   Latitude  \
0        M3A  North York                         Parkwoods  43.753259   
1        M4A  North York                  Victoria Village  43.725882   
2        M6A  North York  Lawrence Manor, Lawrence Heights  43.718518   
3        M3B  North York                         Don Mills  43.745906   
4        M6B  North York                         Glencairn  43.709577   

   Longitude  
0 -79.329656  
1 -79.315572  
2 -79.464763  
3 -79.352188  
4 -79.445073

In [12]:
address = 'North York, Toronto'

geolocator = Nominatim(user_agent="toronto")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of North York are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of North York are 43.7543263, -79.44911696639593.


In [13]:
# Explore and cluster North York, Toronto
NorthYork_map= folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(NorthYork_data['Latitude'], NorthYork_data['Longitude'], NorthYork_data['Borough'], NorthYork_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(NorthYork_map) 
NorthYork_map

In [14]:
CLIENT_ID = '0IOJPD1CR1OVQAUX1YGSFUSZK5P2AVXQLDFNGWHT3DWYRXEN' # your Foursquare ID
CLIENT_SECRET = 'PTLF0Z5VFPV4FJNMZ2ZE303QMNAGSFH3POQHTI2PDPCPNCFD' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 0IOJPD1CR1OVQAUX1YGSFUSZK5P2AVXQLDFNGWHT3DWYRXEN
CLIENT_SECRET:PTLF0Z5VFPV4FJNMZ2ZE303QMNAGSFH3POQHTI2PDPCPNCFD


In [15]:
#Function to apply it to every Postcode we have.
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [19]:
NorthYork_venues = getNearbyVenues(names=NorthYork_data['Neighborhood'],
                                   latitudes=NorthYork_data['Latitude'],
                                   longitudes=NorthYork_data['Longitude']
                                  )

Parkwoods
Victoria Village
Lawrence Manor, Lawrence Heights
Don Mills
Glencairn
Don Mills
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Fairview, Henry Farm, Oriole
Northwood Park, York University
Bayview Village
Downsview
York Mills, Silver Hills
Downsview
North Park, Maple Leaf Park, Upwood Park
Humber Summit
Willowdale, Newtonbrook
Downsview
Bedford Park, Lawrence Manor East
Humberlea, Emery
Willowdale, Willowdale East
Downsview
York Mills West
Willowdale, Willowdale West


In [20]:
print(NorthYork_venues.shape)
NorthYork_venues.head()

(234, 7)


Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0         Parkwoods              43.753259              -79.329656   
1         Parkwoods              43.753259              -79.329656   
2         Parkwoods              43.753259              -79.329656   
3  Victoria Village              43.725882              -79.315572   
4  Victoria Village              43.725882              -79.315572   

                    Venue  Venue Latitude  Venue Longitude  \
0         Brookbanks Park       43.751976       -79.332140   
1          TTC stop #8380       43.752672       -79.326351   
2           Variety Store       43.751974       -79.333114   
3  Victoria Village Arena       43.723481       -79.315635   
4               Portugril       43.725819       -79.312785   

          Venue Category  
0                   Park  
1               Bus Stop  
2      Food & Drink Shop  
3           Hockey Arena  
4  Portuguese Restaurant

Grouping by Neighborhood will be neccesarily, and it is more readable.

In [21]:
NorthYork_venues.groupby('Neighborhood').count().head()

Neighborhood Latitude  \
Neighborhood                                                             
Bathurst Manor, Wilson Heights, Downsview North                     20   
Bayview Village                                                      4   
Bedford Park, Lawrence Manor East                                   23   
Don Mills                                                           26   
Downsview                                                           14   

                                                 Neighborhood Longitude  \
Neighborhood                                                              
Bathurst Manor, Wilson Heights, Downsview North                      20   
Bayview Village                                                       4   
Bedford Park, Lawrence Manor East                                    23   
Don Mills                                                            26   
Downsview                                                            14   

                                                 Venue  Venue Latitude  \
Neighborhood                                                             
Bathurst Manor, Wilson Heights, Downsview North     20              20   
Bayview Village                                      4               4   
Bedford Park, Lawrence Manor East                   23              23   
Don Mills                                           26              26   
Downsview                                           14              14   

                                                 Venue Longitude  \
Neighborhood                                                       
Bathurst Manor, Wilson Heights, Downsview North               20   
Bayview Village                                                4   
Bedford Park, Lawrence Manor East                             23   
Don Mills                                                     26   
Downsview                                                     14   

                                                 Venue Category  
Neighborhood                                                     
Bathurst Manor, Wilson Heights, Downsview North              20  
Bayview Village                                               4  
Bedford Park, Lawrence Manor East                            23  
Don Mills                                                    26  
Downsview                                                    14

In [22]:
print('There are {} uniques categories.'.format(len(NorthYork_venues['Venue Category'].unique())))

There are 100 uniques categories.


Analyzing Each Neighborhood in North York

In [23]:
# one hot encoding
NorthYork_onehot = pd.get_dummies(NorthYork_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
NorthYork_onehot['Neighborhood'] = NorthYork_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [NorthYork_onehot.columns[-1]] + list(NorthYork_onehot.columns[:-1])
NorthYork_onehot = NorthYork_onehot[fixed_columns]
NorthYork_onehot.head()

Neighborhood  Accessories Store  Airport  American Restaurant  \
0         Parkwoods                  0        0                    0   
1         Parkwoods                  0        0                    0   
2         Parkwoods                  0        0                    0   
3  Victoria Village                  0        0                    0   
4  Victoria Village                  0        0                    0   

   Art Gallery  Arts & Crafts Store  Asian Restaurant  Athletics & Sports  \
0            0                    0                 0                   0   
1            0                    0                 0                   0   
2            0                    0                 0                   0   
3            0                    0                 0                   0   
4            0                    0                 0                   0   

   Bakery  Bank  Bar  Baseball Field  Basketball Court  Beer Store  Bike Shop  \
0       0     0    0               0                 0           0          0   
1       0     0    0               0                 0           0          0   
2       0     0    0               0                 0           0          0   
3       0     0    0               0                 0           0          0   
4       0     0    0               0                 0           0          0   

   Boutique  Bridal Shop  Bubble Tea Shop  Burger Joint  Burrito Place  \
0         0            0                0             0              0   
1         0            0                0             0              0   
2         0            0                0             0              0   
3         0            0                0             0              0   
4         0            0                0             0              0   

   Bus Station  Bus Stop  Business Service  Butcher  Café  \
0            0         0                 0        0     0   
1            0         1                 0        0     0   
2            0         0                 0        0     0   
3            0         0                 0        0     0   
4            0         0                 0        0     0   

   Caribbean Restaurant  Chinese Restaurant  Chocolate Shop  Clothing Store  \
0                     0                   0               0               0   
1                     0                   0               0               0   
2                     0                   0               0               0   
3                     0                   0               0               0   
4                     0                   0               0               0   

   Coffee Shop  Comfort Food Restaurant  Construction & Landscaping  \
0            0                        0                           0   
1            0                        0                           0   
2            0                        0                           0   
3            0                        0                           0   
4            0                        0                           0   

   Convenience Store  Cosmetics Shop  Deli / Bodega  Department Store  \
0                  0               0              0                 0   
1                  0               0              0                 0   
2                  0               0              0                 0   
3                  0               0              0                 0   
4                  0               0              0                 0   

   Dim Sum Restaurant  Diner  Discount Store  Distribution Center  Dog Run  \
0                   0      0               0                    0        0   
1                   0      0               0                    0        0   
2                   0      0               0                    0        0   
3                   0      0               0                    0        0   
4                   0      0               0                    0        0   

  

Grouping rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [24]:
NorthYork_grouped = NorthYork_onehot.groupby('Neighborhood').mean().reset_index()
NorthYork_grouped.head()

Neighborhood  Accessories Store  \
0  Bathurst Manor, Wilson Heights, Downsview North                0.0   
1                                  Bayview Village                0.0   
2                Bedford Park, Lawrence Manor East                0.0   
3                                        Don Mills                0.0   
4                                        Downsview                0.0   

    Airport  American Restaurant  Art Gallery  Arts & Crafts Store  \
0  0.000000             0.000000     0.000000                  0.0   
1  0.000000             0.000000     0.000000                  0.0   
2  0.000000             0.043478     0.000000                  0.0   
3  0.000000             0.000000     0.038462                  0.0   
4  0.071429             0.000000     0.000000                  0.0   

   Asian Restaurant  Athletics & Sports  Bakery      Bank  Bar  \
0          0.000000            0.000000     0.0  0.100000  0.0   
1          0.000000            0.000000     0.0  0.250000  0.0   
2          0.000000            0.000000     0.0  0.000000  0.0   
3          0.038462            0.000000     0.0  0.000000  0.0   
4          0.000000            0.071429     0.0  0.071429  0.0   

   Baseball Field  Basketball Court  Beer Store  Bike Shop  Boutique  \
0        0.000000               0.0    0.000000   0.000000       0.0   
1        0.000000               0.0    0.000000   0.000000       0.0   
2        0.000000               0.0    0.000000   0.000000       0.0   
3        0.000000               0.0    0.076923   0.038462       0.0   
4        0.071429               0.0    0.000000   0.000000       0.0   

   Bridal Shop  Bubble Tea Shop  Burger Joint  Burrito Place  Bus Station  \
0         0.05              0.0           0.0            0.0          0.0   
1         0.00              0.0           0.0            0.0          0.0   
2         0.00              0.0           0.0            0.0          0.0   
3         0.00              0.0           0.0            0.0          0.0   
4         0.00              0.0           0.0            0.0          0.0   

   Bus Stop  Business Service   Butcher      Café  Caribbean Restaurant  \
0       0.0          0.000000  0.000000  0.000000              0.000000   
1       0.0          0.000000  0.000000  0.250000              0.000000   
2       0.0          0.000000  0.043478  0.043478              0.000000   
3       0.0          0.000000  0.000000  0.038462              0.038462   
4       0.0          0.071429  0.000000  0.000000              0.000000   

   Chinese Restaurant  Chocolate Shop  Clothing Store  Coffee Shop  \
0            0.000000             0.0        0.000000     0.100000   
1            0.250000             0.0        0.000000     0.000000   
2            0.000000             0.0        0.000000     0.086957   
3            0.038462             0.0        0.038462     0.076923   
4            0.000000             0.0        0.000000     0.000000   

   Comfort Food Restaurant  Construction & Landscaping  Convenience Store  \
0                 0.000000                         0.0                0.0   
1                 0.000000                         0.0                0.0   
2                 0.043478                         0.0                0.0   
3                 0.000000                         0.0                0.0   
4                 0.000000                         0.0                0.0   

   Cosmetics Shop  Deli / Bodega  Department Store  Dim Sum Restaurant  Diner  \
0             0.0           0.05               0.0            0.000000   0.05   
1             0.0           0.00               0.0            0.000000   0.00   
2             0.0           0.00               0.0            0.000000   0.00   
3             0.0           0.00               0.0            0.038462   0.00   
4             0.0           0.00               0.0            0.000000   0.00   

   Discount Store  Distribution Center  Dog Run  Electronics St

Printing each neighborhood along with the top 5 most common venues

In [25]:
num_top_venues = 5

for hood in NorthYork_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = NorthYork_grouped[NorthYork_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor, Wilson Heights, Downsview North----
               venue  freq
0        Coffee Shop  0.10
1               Bank  0.10
2  Health Food Store  0.05
3      Shopping Mall  0.05
4  Mobile Phone Shop  0.05


----Bayview Village----
                 venue  freq
0   Chinese Restaurant  0.25
1                 Café  0.25
2                 Bank  0.25
3  Japanese Restaurant  0.25
4         Liquor Store  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0         Coffee Shop  0.09
1      Sandwich Place  0.09
2  Italian Restaurant  0.09
3         Pizza Place  0.04
4             Butcher  0.04


----Don Mills----
                 venue  freq
0                  Gym  0.12
1          Coffee Shop  0.08
2           Restaurant  0.08
3  Japanese Restaurant  0.08
4           Beer Store  0.08


----Downsview----
           venue  freq
0  Grocery Store  0.21
1           Park  0.14
2     Food Truck  0.07
3  Shopping Mall  0.07
4        Airport  0.07


----Fairview, He